In [37]:
import pandas as pd
import numpy as np

In [38]:
# Load the Dataset
all_listings = pd.read_csv('/Users/saphir/Desktop/UChi/DE/Final Project/Airbnb/Data/all_listings.csv')
all_listings.head()

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license,quarter
0,2384,Hyde Park - Walk to UChicago,2613,Rebecca,Hyde Park,41.78790,-87.58780,Private room,90,3,212,04/03/2023,2.13,1,347,20,R17000015609,Q1
1,94450,Comfy Garden Suite in Andersonville,504470,Mark,Edgewater,41.97960,-87.66512,Entire home/apt,65,7,7,01/12/2022,0.88,1,178,7,R22000082956,Q1
2,145659,Trendy Roscoe Village 3BR/2BR walk to shops,683529,Joe,North Center,41.94342,-87.68121,Entire home/apt,198,2,59,31/12/2022,0.65,22,330,15,2209272,Q1
3,7126,Tiny Studio Apartment 94 Walk Score,17928,Sarah,West Town,41.90166,-87.68021,Entire home/apt,85,2,483,26/02/2023,2.89,1,279,48,R21000075737,Q1
4,189821,"Best in Chicago, private, amazing garden space",899757,Meighan,Logan Square,41.92918,-87.70219,Entire home/apt,202,3,598,25/02/2023,4.27,1,207,24,R21000062936,Q1


In [39]:
# Clean the price outliers
all_listings = all_listings[all_listings['price']<20000]
all_listings = all_listings[all_listings['price']>0]

In [40]:
# Normalization for room_type
room_types = all_listings['room_type'].unique()
room_type_df = pd.DataFrame(room_types, columns=['room_type'])
room_type_df.insert(0, 'room_id', range(1, len(room_type_df) + 1))
room_type_df.head()

,room_id,room_type
0,1,Private room
1,2,Entire home/apt
2,3,Shared room
3,4,Hotel room


In [41]:
# Normalization for license
def categorize_license(license_val):
    if pd.isnull(license_val):
        return "No license"
    elif license_val == "City registration pending":
        return "Pending"
    else:
        return "Licensed"

all_listings['license_status'] = all_listings['license'].apply(categorize_license)
licenses = all_listings['license_status'].unique()
license_df = pd.DataFrame(licenses, columns=['license_status'])
license_df.insert(0, 'license_id', range(1, len(license_df) + 1))
license_df

,license_id,license_status
0,1,Licensed
1,2,No license
2,3,Pending


In [42]:
# Normalization for the host table
# Creating a mapping of host_id to host_listing_count and then map it to the host_df
host_listing_count = all_listings.groupby('host_id').size()
host_df = all_listings[['host_id', 'host_name']].drop_duplicates().set_index('host_id')
host_df['host_listing_count'] = host_listing_count

# Reset index to make 'host_id' a column again
host_df = host_df.reset_index()
host_df

,host_id,host_name,host_listing_count
0,2613,Rebecca,4
1,504470,Mark,4
2,683529,Joe,90
3,17928,Sarah,4
4,899757,Meighan,4
...,...,...,...
4609,449750923,Danielle,1
4610,489678213,Megan,1
4611,59721414,Tiffany,1
4612,490950442,Oguzhan,1


In [43]:
# Load the neighbourhood dataset
neighbourhood_df = pd.read_csv('/Users/saphir/Desktop/UChi/DE/Final Project/Airbnb/Data/neighbourhood.csv')
neighbourhood_df = neighbourhood_df.rename(columns={'community_area': 'neighbourhood_id', 'name': 'neighbourhood'})
neighbourhood_df

,neighbourhood_id,neighbourhood
0,1,Rogers Park
1,2,West Ridge
2,3,Uptown
3,4,Lincoln Square
4,5,North Center
...,...,...
72,73,Washington Heights
73,74,Mount Greenwood
74,75,Morgan Park
75,76,O'Hare


In [44]:
# Create mappings for room_type, neighbourhood, and license
room_type_mapping = room_type_df.set_index('room_type').to_dict()['room_id']
neighbourhood_mapping = neighbourhood_df.set_index('neighbourhood').to_dict()['neighbourhood_id']
license_mapping = license_df.set_index('license_status').to_dict()['license_id']

# Normalize the 'quarter' column in the listings
quarter_mapping = {'Q1': 1, 'Q2': 2, 'Q3': 3, 'Q4': 4}
all_listings['quarter'] = all_listings['quarter'].map(quarter_mapping).astype('int')

# Transform 'availability_365'column to 'occupancy'
all_listings['occupancy'] = 365 - all_listings['availability_365']

# Construct the normalized listing table
listing_columns = {
    'id': 'listing_id',
    'host_id': 'host_id',
    'roomtype_id': 'roomtype_id',
    'neighbourhood_id': 'neighbourhood_id',
    'license_id': 'license_id',
    'price': 'avg_price',
    'minimum_nights': 'min_nights',
    'reviews_per_month': 'review_per_month',
    'occupancy': 'occupancy',
    'quarter': 'quarter'
}
all_listings['roomtype_id'] = all_listings['room_type'].map(room_type_mapping)
all_listings['neighbourhood_id'] = all_listings['neighbourhood'].map(neighbourhood_mapping)
all_listings['license_id'] = all_listings['license_status'].map(license_mapping)
listing_df = all_listings[list(listing_columns.keys())].rename(columns=listing_columns)
listing_df

,listing_id,host_id,roomtype_id,neighbourhood_id,license_id,avg_price,min_nights,review_per_month,occupancy,quarter
0,2384,2613,1,41.0,1,90,3,2.13,18,1
1,94450,504470,2,77.0,1,65,7,0.88,187,1
2,145659,683529,2,5.0,1,198,2,0.65,35,1
3,7126,17928,2,24.0,1,85,2,2.89,86,1
4,189821,899757,2,22.0,1,202,3,4.27,158,1
...,...,...,...,...,...,...,...,...,...,...
31919,771880753874742770,488113016,2,2.0,1,66,1,0.00,295,4
31920,771881329605160225,488113016,2,1.0,1,92,1,0.00,299,4
31921,779992322133479260,490950442,2,8.0,2,33,60,0.00,256,4
31922,780097517605219739,316658141,1,32.0,1,143,4,0.00,307,4


In [45]:
# Saving normalized tables
room_type_df.to_csv('/Users/saphir/Desktop/UChi/DE/Final Project/Airbnb/Data/Normalization/room_type.csv', index=False)
neighbourhood_df.to_csv('/Users/saphir/Desktop/UChi/DE/Final Project/Airbnb/Data/Normalization/neighbourhood.csv', index=False)
license_df.to_csv('/Users/saphir/Desktop/UChi/DE/Final Project/Airbnb/Data/Normalization/license.csv', index=False)
host_df.to_csv('/Users/saphir/Desktop/UChi/DE/Final Project/Airbnb/Data/Normalization/host.csv', index=False)
listing_df.to_csv('/Users/saphir/Desktop/UChi/DE/Final Project/Airbnb/Data/Normalization/listing.csv', index=False)